In [ ]:
from utilities import * 
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%load_ext autoreload
%autoreload 2

key = os.getenv('AZURE_KEY')
endpoint = os.environ.get('AZURE_ENDPOINT')

## Extract information using Azure:

In [ ]:
document_analysis_client = DocumentAnalysisClient(endpoint=endpoint, 
                                                  credential=AzureKeyCredential(key))

In [ ]:
result_dict = {}
captured_errors = []
error_list = []

for file_path in tqdm(glob.glob('../dataset/raw/nist/nist_data/*.png')):
    
    image_name = file_path[file_path.rfind('/')+1:]
    
    try:
        with open(file_path, "rb") as f:
            poller = document_analysis_client.begin_analyze_document("prebuilt-document", 
                                                                     document=f)
        result = poller.result()

        dict__ = {}
        for i in result.key_value_pairs:
            if i.to_dict()['value'] != None:
                dict__[i.to_dict()['key']['content']] = {
                    'value': i.to_dict()['value']['content'],
                    'confidence':i.to_dict()['confidence'] }


        result_dict[image_name] = {'result_summary': dict__, 
                                   'result_full': result}
    except Exception as e:
        time.sleep(30)
        error_list.append(e)
        captured_errors.append(file_path)
        

## save resutls

In [ ]:
with open('../dataset/raw/nist/full_azure_form.pkl', 'wb') as fp:
    pickle.dump(result_dict, fp)